In [ ]:
import os
from dotenv import load_dotenv
import mlflow

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
if os.path.exists(".env"):
    load_dotenv()  # Load environment variables from .env if available
    print("Loaded .env file for local development")
    
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
os.environ.get("MLFLOW_TRACKING_USERNAME")
os.environ.get("MLFLOW_TRACKING_PASSWORD")

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/ArtemStrashko/end_to_end_mlops_project.mlflow",
           
        )

        return model_evaluation_config


In [ ]:
import os
import pandas as pd
from sklearn.metrics import classification_report
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self, y_true, y_pred):
        # Initialize result dictionary
        results = {}

        # Precision, recall, and F1-score
        class_report = classification_report(y_true, y_pred, output_dict=True)
        results["precision"] = class_report["macro avg"]["precision"]
        results["recall"] = class_report["macro avg"]["recall"]
        results["f1_score"] = class_report["macro avg"]["f1-score"]

        return results
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_labels = model.predict(test_x)
            scores = self.eval_metrics(y_true = test_y, y_pred = predicted_labels)
            
            # Saving metrics as local
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("precision", scores["precision"])
            mlflow.log_metric("recall", scores["recall"])
            mlflow.log_metric("f1_score", scores["f1_score"])


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e